In [ ]:
load into Snowflake for analytical work

But first go through code line by line to fully understand what I have done

In [1]:
import requests, yaml, pandas as pd
from pathlib import Path
from io import StringIO  # To handle CSV data in memory

def get_config():
    # Load the YAML file
    with open('config.yml', 'r') as file:
        config = yaml.safe_load(file)
    return config
config = get_config()

# Global constants
API_KEY = config['api']['api_key']
dbname = config['DB_NAME']
user = config['DB_USER']
password = config['DB_PASSWORD']
host = config['DB_HOST']
port = config['DB_PORT']


def auntenticate(API_KEY):
    # Request for S&P 500 list (this endpoint gives you the entire listing)
    url = f"https://www.alphavantage.co/query?function=LISTING_STATUS&apikey={API_KEY}"

    # Make the API request
    response = requests.get(url)
    
    return response

def run_request_extract(response):
    # Check if the request was successful
    if response.status_code == 200:
        try:
            # Since the data is CSV, we'll use StringIO to read it into a DataFrame
            csv_data = StringIO(response.text)  # Convert the string response to a file-like object
            
            # Load the CSV data into a DataFrame
            df = pd.read_csv(csv_data)
            
            # Display the sample 10 rows of the DataFrame in raw format
            #print(df.sample(10))
        
        except ValueError as e:
            # Handle error if unable to read the CSV
            print("Error reading CSV:", e)
            print("Response text:", response.text)  # Inspect the actual response content
    else:
        print(f"Error: Received status code {response.status_code}")
        print("Response content:", response.text)  # Inspect the raw response content
    return df

def transformation(df):
    df.columns = df.columns.str.strip()  # Strip any leading/trailing spaces from column names
    df = df.drop('delistingDate', axis=1, errors='ignore')  # Avoid errors if column isn't found
    df['ipoDate'] = pd.to_datetime(df['ipoDate'], errors='coerce') # Turns ipoDate from a str object into a datetime64
    return df

def csv_output(df):
    filepath = Path()
    filename = 'transformed_cleaned.csv'
    
    output_dir = df.to_csv(filepath / filename)
    
    return df, output_dir

def load_snowflake():
    pass 

def main():
    first_auth = auntenticate(API_KEY)
    requests_extract = run_request_extract(first_auth)
    t_etl = transformation(requests_extract)
    csv_excel = csv_output(t_etl)

    print(csv_excel)

import psycopg2

try:
    # Connect to PostgreSQL
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    print("✅ Connected to PostgreSQL!")

    # Create a cursor object
    cur = conn.cursor()

except Exception as e:
    print(f"❌ Connection error: {e}")


if __name__ == '__main__':
    main()

❌ Connection error: name 'DB_NAME' is not defined
(      symbol                                               name   exchange  \
0          A                           Agilent Technologies Inc       NYSE   
1         AA                                         Alcoa Corp       NYSE   
2        AAA    ALTERNATIVE ACCESS FIRST PRIORITY CLO BOND ETF   NYSE ARCA   
3       AAAU                    Goldman Sachs Physical Gold ETF       BATS   
4      AACBU  Artius II Acquisition Inc - Units (1 Ord Shs &...     NASDAQ   
...      ...                                                ...        ...   
11812   ZYME                                   Zymeworks BC Inc     NASDAQ   
11813   ZYNE                        Zynerba Pharmaceuticals Inc     NASDAQ   
11814   ZYXI                                          Zynex Inc     NASDAQ   
11815    ZZK                                                NaN  NYSE ARCA   
11816    ZZZ                                TEST TICKER FOR UTP  NYSE ARCA   

      assetT